In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display

import psycopg2
from netrc import netrc
from fiona.crs import from_epsg

user, acct, passwd = netrc().authenticators("harris")

seats, fips, epsg = 8, 55, 3695

%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000

In [27]:
wif = "/media/jsaxon/brobdingnag/data/el_dv/wi_site/"
map_04 = gpd.read_file(wif + "2004/wtm_8391_wards_2004_ed.shp")
map_04.rename(columns = {"PRES_DEM" : "D04", "PRES_REP" : "R04"}, inplace = True)
map_04 = map_04[["D04", "R04", "geometry"]]
map_04 = map_04.to_crs(epsg = 3695)

map_08 = gpd.read_file(wif + "2008/2008_Election_Data_By_Ward.shp")
map_08.rename(columns = {"PRESDEM08" : "D08", "PRESREP08" : "R08"}, inplace = True)
map_08 = map_08[["D08", "R08", "geometry"]]
map_08 = map_08.to_crs(epsg = 3695)

map_12 = gpd.read_file(wif + "2012/Wards_111312_ED_110612.shp")
map_12.rename(columns = {"PRES_DEM12" : "D12", "PRES_REP12" : "R12"}, inplace = True)
map_12 = map_12[["D12", "R12", "geometry"]]
map_12 = map_12.to_crs(epsg = 3695)

In [28]:
query = """SELECT 
              rn, ST_Centroid(ST_Transform(tr.geom, epsg)) geometry
           FROM census_tracts_2015 AS tr
           JOIN (SELECT state, county, tract,
                        row_number() over 
                          (PARTITION BY state ORDER BY county, tract NULLS LAST) - 1 as rn
                 FROM census_tracts_2015) rn ON
                   tr.state  = rn.state  AND
                   tr.county = rn.county AND
                   tr.tract  = rn.tract
           JOIN states AS st ON st.fips = tr.state
           WHERE tr.state = {} ORDER BY rn;
           """

con = psycopg2.connect(database = "census", user = user, password = passwd,
                       host = "saxon.harris.uchicago.edu", port = 5432)

tr_rn = gpd.GeoDataFrame.from_postgis(query.format(fips), con, index_col = "rn",
                                      geom_col = "geometry", crs = from_epsg(epsg))

In [34]:
def merge_tract_number(gdf, trdf):

    gdf["rn"] = float('nan')

    for pri, row in gdf.iterrows():
        ctr = row.geometry.centroid
        distances = [(xi, pt.distance(ctr)) for xi, pt in enumerate(tr_rn.geometry)]
        match = min(distances, key=lambda item:item[1])
        gdf.loc[pri, "rn"] = match[0]
        if (pri % 100) == 0: print(pri, end = " ")
            
    print()

merge_tract_number(map_04, tr_rn)
merge_tract_number(map_08, tr_rn)
merge_tract_number(map_12, tr_rn)

map_04 = map_04.groupby("rn").sum().filter(regex = '[RD][01][248]')
map_08 = map_08.groupby("rn").sum().filter(regex = '[RD][01][248]')
map_12 = map_12.groupby("rn").sum().filter(regex = '[RD][01][248]')

In [41]:
wi_votes = pd.concat([map_04, map_08, map_12], axis = 1).fillna(0)
wi_votes.to_csv("wi_votes.csv")
wi_votes.head()

,D04,R04,D08,R08,D12,R12
rn,,,,,,
0.0,903.0,1102.0,954.0,1015.0,876.0,1127.0
1.0,962.0,809.0,1037.0,643.0,1022.0,789.0
2.0,620.0,465.0,655.0,374.0,449.0,342.0
3.0,830.0,664.0,903.0,462.0,988.0,680.0
4.0,419.0,287.0,428.0,205.0,497.0,253.0
